## Scripts Downloader
#### May 2022, Ziyi

#### Input: save file path & literature csv
#### Make sure installed required packages

In [18]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

#### Specify dowload destination & Specify download query below 

In [6]:
# change before running cells below
sPath = ('E:/test')
# make new directory if the path not exist
if os.path.exists(sPath) == False:
    os.mkdir(sPath)
searchResult = 'csv-DivergentT-set.csv'
df = pd.read_csv(searchResult)

In [7]:
# read the input file & retrive the doi
doi = pd.read_csv(searchResult)['DOI']
doi_list = doi.tolist()

#### Personalize save file name: FirstAuthorLastName_PublicationYear 
#### Eg. Beaty_2016

In [8]:
# Personalize save file name
# comment out if you don't want it
saveNames = []
for index in range(df.shape[0]):
    spaceIndex = df['First Author'][index].find(' ')
    firstAuthor = df['First Author'][index][:spaceIndex]
    year = df['Publication Year'][index]
    saveNames.append(firstAuthor+ '_' + str(year))

#### This is a input sample

In [9]:
df

,PMID,Title,Authors,Citation,First Author,Journal/Book,Publication Year,Create Date,PMCID,NIHMS ID,DOI
0,26553223,Creative Cognition and Brain Network Dynamics,"Beaty RE, Benedek M, Silvia PJ, Schacter DL.",Trends Cogn Sci. 2016 Feb;20(2):87-95. doi: 10...,Beaty RE,Trends Cogn Sci,2016,2015/11/11,PMC4724474,NIHMS731920,10.1016/j.tics.2015.10.004
1,32882380,The relationship between methods of scoring th...,"Vartanian O, Smith I, Lam TK, King K, Lam Q, B...",Neuroimage. 2020 Dec;223:117325. doi: 10.1016/...,Vartanian O,Neuroimage,2020,2020/09/04,NaN,NaN,10.1016/j.neuroimage.2020.117325
2,11316014,Risk as feelings,"Loewenstein GF, Weber EU, Hsee CK, Welch N.",Psychol Bull. 2001 Mar;127(2):267-86. doi: 10....,Loewenstein GF,Psychol Bull,2001,2001/04/24,NaN,NaN,10.1037/0033-2909.127.2.267
3,33035524,Creativity and ADHD: A review of behavioral st...,"Hoogman M, Stolte M, Baas M, Kroesbergen E.",Neurosci Biobehav Rev. 2020 Dec;119:66-85. doi...,Hoogman M,Neurosci Biobehav Rev,2020,2020/10/09,NaN,NaN,10.1016/j.neubiorev.2020.09.029
4,30676427,The Relationship Between Critical Thinking Ski...,"Shirazi F, Heidari S.",J Nurs Res. 2019 Aug;27(4):e38. doi: 10.1097/j...,Shirazi F,J Nurs Res,2019,2019/01/25,PMC6641090,NaN,10.1097/jnr.0000000000000307
5,33328929,Think Hard or Think Smart: Network Reconfigura...,"Wu HY, Kuo BC, Huang CM, Tsai PJ, Hsu AL, Hsu ...",Front Hum Neurosci. 2020 Nov 20;14:571118. doi...,Wu HY,Front Hum Neurosci,2020,2020/12/17,PMC7714934,NaN,10.3389/fnhum.2020.571118
6,30128937,Types of creativity,Dietrich A.,Psychon Bull Rev. 2019 Feb;26(1):1-12. doi: 10...,Dietrich A,Psychon Bull Rev,2019,2018/08/22,NaN,NaN,10.3758/s13423-018-1517-7
7,34182887,Divergent thinking and constructing future eve...,"Thakral PP, Yang AC, Addis DR, Schacter DL.",Memory. 2021 Jul;29(6):729-743. doi: 10.1080/0...,Thakral PP,Memory,2021,2021/06/29,PMC8414037,NIHMS1714210,10.1080/09658211.2021.1940205
8,32729407,The Opposite of Stress,"Meier M, Unternaehrer E, Schorpp SM, Wenzel M,...",Exp Psychol. 2020 Mar;67(2):150-159. doi: 10.1...,Meier M,Exp Psychol,2020,2020/07/31,NaN,NaN,10.1027/1618-3169/a000483
9,30542311,Use or Consequences: Probing the Cognitive Dif...,"Hass RW, Beaty RE.",Front Psychol. 2018 Nov 27;9:2327. doi: 10.338...,Hass RW,Front Psychol,2018,2018/12/14,PMC6278612,NaN,10.3389/fpsyg.2018.02327


In [10]:
## Pre-defined Headers
HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:27.0) Gecko/20100101 Firefox/27.0'}

In [17]:
lostDOI = []
# keep track papers not included on sci-hub

for iteration, doi in enumerate(doi_list):
    if doi is None:
        print('------ '+ doi + ' Not Found! -----')
    else:
        url = 'https://www.sci-hub.ren/' + doi
        print('downloading ' + doi)
        # the download url
        res = requests.get(url, headers=HEADERS)
        # add header to prevent Error 403, update the header if it fails
        soup = BeautifulSoup(res.text, 'html.parser')
        # transforming the initial url to a readable file to get iframe
        if soup.iframe == None:
            print('--- Paper Not Found on Sci-Hub ---')
            lostDOI.append(doi)
            continue 
        else:
            rUrl = soup.iframe.attrs['src']
            print(rUrl)
            # the actual script is embedded in a iframe, we need to access the real url before downloading
            pdf = requests.get(rUrl, headers = HEADERS)
            pdf.raise_for_status()
            # download the actual script
            OutFile = sPath + '/' + str(saveNames[iteration])+ '.pdf'
            with open('%s.pdf' % (OutFile), 'wb') as f:
                f.write(pdf.content)
            # save the file to designated path
    print('--- Download Complete ---')

print(lostDOI)

downloading 10.1016/j.tics.2015.10.004
https://sci.bban.top/pdf/10.1016/j.tics.2015.10.004.pdf#view=FitH
--- Download Complete ---
downloading 10.1016/j.neuroimage.2020.117325
https://sci.bban.top/pdf/10.1016/j.neuroimage.2020.117325.pdf#view=FitH
--- Download Complete ---
downloading 10.1037/0033-2909.127.2.267
https://sci.bban.top/pdf/10.1037/0033-2909.127.2.267.pdf#view=FitH
--- Download Complete ---
downloading 10.1016/j.neubiorev.2020.09.029
https://sci.bban.top/pdf/10.1016/j.neubiorev.2020.09.029.pdf#view=FitH
--- Download Complete ---
downloading 10.1097/jnr.0000000000000307
https://sci.bban.top/pdf/10.1097/jnr.0000000000000307.pdf#view=FitH
--- Download Complete ---
downloading 10.3389/fnhum.2020.571118
https://sci.bban.top/pdf/10.3389/fnhum.2020.571118.pdf#view=FitH
--- Download Complete ---
downloading 10.3758/s13423-018-1517-7
https://sci.bban.top/pdf/10.3758/s13423-018-1517-7.pdf#view=FitH
--- Download Complete ---
downloading 10.1080/09658211.2021.1940205
https://sci.bban.